In [1]:
import json
import pandas as pd
path = "/Users/acabrera/dev/data/coco-2014/annotations/instances_val2014.json"
anns = json.load(open(path))
import numpy as np

In [2]:
df_annot = pd.DataFrame(anns["annotations"])
df_annot = df_annot.drop("id", axis=1)
df_annot["id"] = df_annot["image_id"]
df_annot = df_annot.drop("image_id", axis=1)

In [3]:
df_ids = pd.DataFrame(anns["images"])

In [4]:
df_annot.head()

,segmentation,area,iscrowd,bbox,category_id,id
0,"[[239.97, 260.24, 222.04, 270.49, 199.84, 253....",2765.14865,0,"[199.84, 200.46, 77.71, 70.88]",58,558840
1,"[[247.71, 354.7, 253.49, 346.99, 276.63, 337.3...",1545.42130,0,"[234.22, 317.11, 149.39, 38.55]",58,200365
2,"[[274.58, 405.68, 298.32, 405.68, 302.45, 402....",5607.66135,0,"[239.48, 347.87, 160.0, 57.81]",58,200365
3,"[[296.65, 388.33, 296.65, 388.33, 297.68, 388....",0.00000,0,"[296.65, 388.33, 1.03, 0.0]",58,200365
4,"[[251.87, 356.13, 260.13, 343.74, 300.39, 335....",800.41325,0,"[251.87, 333.42, 125.94, 22.71]",58,200365


In [5]:
df_ids.head()

,license,file_name,coco_url,height,width,date_captured,flickr_url,id
0,3,COCO_val2014_000000391895.jpg,http://images.cocodataset.org/val2014/COCO_val...,360,640,2013-11-14 11:18:45,http://farm9.staticflickr.com/8186/8119368305_...,391895
1,4,COCO_val2014_000000522418.jpg,http://images.cocodataset.org/val2014/COCO_val...,480,640,2013-11-14 11:38:44,http://farm1.staticflickr.com/1/127244861_ab0c...,522418
2,3,COCO_val2014_000000184613.jpg,http://images.cocodataset.org/val2014/COCO_val...,336,500,2013-11-14 12:36:29,http://farm3.staticflickr.com/2169/2118578392_...,184613
3,3,COCO_val2014_000000318219.jpg,http://images.cocodataset.org/val2014/COCO_val...,640,556,2013-11-14 13:02:53,http://farm5.staticflickr.com/4125/5094763076_...,318219
4,3,COCO_val2014_000000554625.jpg,http://images.cocodataset.org/val2014/COCO_val...,640,426,2013-11-14 16:03:19,http://farm5.staticflickr.com/4086/5094162993_...,554625


In [6]:
df_grouped = df_annot[["id", "area", "bbox", "category_id"]].groupby("id")

In [7]:
agg = df_grouped.agg(list)

In [8]:
print(agg.shape)
print(df_ids.shape)

(40137, 3)
(40504, 8)


In [102]:
joined = df_ids.join(agg, on="id")
joined = joined[(joined["bbox"].isnull() == False) & (joined["category_id"].isnull() == False)]

In [103]:
def map_bbox(d):
    return np.concatenate((np.array(d["bbox"]), np.array(d["category_id"])[:,None]), axis=1).tolist()
joined["label"] = joined.apply(map_bbox, axis=1)

In [105]:
joined = joined.drop("bbox", axis=1)
joined = joined.drop("category_id", axis=1)

In [106]:
joined.to_parquet('coco.parquet')

In [107]:
loaded = pd.read_parquet('coco.parquet')

In [108]:
loaded.tail()

,license,file_name,coco_url,height,width,date_captured,flickr_url,id,area,label
40499,5,COCO_val2014_000000134574.jpg,http://images.cocodataset.org/val2014/COCO_val...,480,640,2013-11-25 14:27:46,http://farm8.staticflickr.com/7316/9531042830_...,134574,"[240805.90164999999, 16074.983200000002, 5056....","[[0.0, 58.32, 637.48, 410.97, 67.0], [304.18, ..."
40500,1,COCO_val2014_000000572233.jpg,http://images.cocodataset.org/val2014/COCO_val...,427,640,2013-11-25 14:48:33,http://farm4.staticflickr.com/3751/9300793319_...,572233,"[2510.9476999999993, 213.35485000000006, 30749...","[[353.11, 332.96, 69.09, 94.04, 62.0], [590.77..."
40501,1,COCO_val2014_000000418825.jpg,http://images.cocodataset.org/val2014/COCO_val...,640,480,2013-11-25 15:04:26,http://farm8.staticflickr.com/7373/9111627012_...,418825,"[26308.171299999998, 7319.351899999999, 5639.2...","[[2.28, 0.0, 204.71, 253.51, 64.0], [396.98, 9..."
40502,1,COCO_val2014_000000560744.jpg,http://images.cocodataset.org/val2014/COCO_val...,480,640,2013-11-25 15:04:29,http://farm4.staticflickr.com/3791/9109408773_...,560744,"[3841.1774000000005, 1491.2382999999973, 85225...","[[444.27, 244.51, 40.44, 137.94, 44.0], [550.4..."
40503,3,COCO_val2014_000000074478.jpg,http://images.cocodataset.org/val2014/COCO_val...,640,428,2013-11-25 20:09:45,http://farm3.staticflickr.com/2753/4318988969_...,74478,"[2444.77515, 1963.5344000000005, 1848.64565000...","[[0.72, 491.05, 33.55, 148.95, 44.0], [6.54, 4..."
